# Nuclio Serverless on Amazon EKS with Hydraa

This notebook demonstrates how to deploy and manage serverless functions on Nuclio using Amazon EKS.

## Setup and Initialization

In [ ]:
import os
import json
import time
import traceback
from hydraa import proxy, AWS, Task, AwsVM
from hydraa.services import ServiceManager
from hydraa_faas.faas_manager.manager import FaasManager

In [ ]:
# Check for required environment variables
required_env_vars = ['ACCESS_KEY_ID', 'ACCESS_KEY_SECRET', 'AWS_REGION']
missing_vars = [var for var in required_env_vars if not os.environ.get(var)]
if missing_vars:
    print(f"Missing environment variables: {missing_vars}")
    print("Please set them before running this notebook")
    print("Example:")
    print("export ACCESS_KEY_ID=your_access_key")
    print("export ACCESS_KEY_SECRET=your_secret_key")
    print("export AWS_REGION=us-east-1")
else:
    print("All required environment variables are set")

In [ ]:
# Create AWS VM for EKS deployment
vm = AwsVM(
    launch_type='EKS',
    instance_id='t3.medium',
    min_count=2,
    max_count=4,
    image_id='ami-0abcdef1234567890'  # EKS optimized AMI - replace with actual AMI ID for your region
)

# Configure ECR for container registry
registry_config = {
    'nuclio': {
        'registry': {
            'registry_type': 'ecr',
            'registry_region': os.environ.get('AWS_REGION', 'us-east-1'),
            'auto_setup_resources': True
        },
        'shared_repository': True,  # Use shared repository for all functions
        'enable_scale_to_zero': True
    }
}

# Initialize proxy and FaaS manager
provider_mgr = proxy([AWS])

try:
    faas_mgr = FaasManager(
        proxy_mgr=provider_mgr,
        vms=[vm],
        asynchronous=False,
        auto_terminate=True,  # Clean up EKS cluster on shutdown
        resource_config={'aws': registry_config}
    )

    # Start services
    service_mgr = ServiceManager([faas_mgr])
    service_mgr.start_services()

    # Wait for EKS cluster and Nuclio initialization
    print("Waiting for EKS cluster and Nuclio to be ready...")
    print("This may take 10-15 minutes for initial setup...")
    time.sleep(120)  # Initial wait for EKS
    
    # Check status
    status = faas_mgr.get_status()
    print(f"FaaS Manager Status: {json.dumps(status, indent=2)}")
    
except Exception as e:
    print(f"Failed to initialize FaaS Manager: {e}")
    traceback.print_exc()

## 1. Inline Code Deployment

In [ ]:
# Simple inline function example
inline_task = Task()
inline_task.provider = 'nuclio'
inline_task.cloud_provider = 'aws'  # Hint for multi-cloud support
inline_task.memory = 256
inline_task.vcpus = 0.25

# Add runtime attributes
inline_task.runtime = 'python:3.9'
inline_task.handler = 'main:handler'
inline_task.handler_code = '''
import json
import os
import boto3

def handler(context, event):
    body = event.body
    if isinstance(body, bytes):
        body = body.decode('utf-8')
    
    data = {}
    if body:
        try:
            data = json.loads(body)
        except json.JSONDecodeError:
            pass
    
    # Get AWS region from environment
    region = os.environ.get('AWS_REGION', 'unknown')
    
    return json.dumps({
        'message': 'Hello from Nuclio on EKS!',
        'cluster': 'Amazon EKS',
        'region': region,
        'received': data
    })
'''

# Add build commands for AWS SDK
inline_task.build_commands = ['pip install boto3']

try:
    # Deploy
    print("Deploying inline function...")
    faas_mgr.submit(inline_task)
    result = inline_task.result()
    print(f"Inline function deployed: {result}")
    
    # Wait for function to be ready
    time.sleep(10)

    # Test - use the nuclio-aws provider name
    response = faas_mgr.invoke(inline_task.name, {'environment': 'eks'}, provider='nuclio-aws')
    print(f"Response: {response}")
except Exception as e:
    print(f"Deployment failed: {e}")
    traceback.print_exc()

## 2. Source Directory Deployment with ECR

In [ ]:
# Create directory structure
import os
os.makedirs('./nuclio_functions/image_classifier', exist_ok=True)
os.makedirs('./nuclio_functions/stream_processor', exist_ok=True)

In [ ]:
# Deploy ML inference function that builds and pushes to ECR
ml_task = Task()
ml_task.provider = 'nuclio'
ml_task.cloud_provider = 'aws'
ml_task.memory = 2048
ml_task.vcpus = 1

# Add attributes
ml_task.runtime = 'python:3.9'
ml_task.source_path = './nuclio_functions/image_classifier'
ml_task.handler = 'classifier:classify'
ml_task.env_vars = {
    'MODEL_TYPE': 'resnet50',
    'BATCH_SIZE': '32',
    'AWS_REGION': os.environ.get('AWS_REGION', 'us-east-1')
}
ml_task.min_replicas = 2
ml_task.max_replicas = 10

# Add build commands for ML dependencies
ml_task.build_commands = [
    'pip install pillow',
    'pip install requests',
    'pip install numpy',
    'pip install boto3',
    'pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu'
]

try:
    # Deploy
    print("Deploying ML function with ECR image...")
    faas_mgr.submit(ml_task)
    result = ml_task.result()
    print(f"ML function deployed: {result}")
    
    time.sleep(15)  # Give more time for larger deployment

    # Test
    test_batch = {
        'images': ['https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg'],
        'return_probabilities': True
    }
    response = faas_mgr.invoke(ml_task.name, test_batch, provider='nuclio-aws')
    print(f"Classification results: {json.dumps(response, indent=2)}")
except Exception as e:
    print(f"ML deployment failed: {e}")
    traceback.print_exc()

In [ ]:
# Deploy stream processor with auto-scaling
stream_task = Task()
stream_task.provider = 'nuclio'
stream_task.cloud_provider = 'aws'
stream_task.memory = 512
stream_task.vcpus = 0.5

# Add attributes
stream_task.runtime = 'python:3.9'
stream_task.source_path = './nuclio_functions/stream_processor'
stream_task.handler = 'processor:process_stream'
stream_task.min_replicas = 1
stream_task.max_replicas = 20  # Higher max for cloud scaling

# Add build commands
stream_task.build_commands = [
    'pip install boto3',
    'pip install uuid'
]

# Add environment variables for AWS integration
stream_task.env_vars = {
    'ENABLE_STORAGE': 'true',
    'STORAGE_TYPE': 's3',
    'S3_BUCKET': 'nuclio-stream-results'  # Replace with your bucket
}

try:
    # Deploy
    print("Deploying stream processor...")
    faas_mgr.submit(stream_task)
    result = stream_task.result()
    print(f"Stream processor deployed: {result}")
    
    time.sleep(10)

    # Test
    test_events = {
        'events': [
            {'id': 1, 'type': 'order', 'amount': 99.99, 'customer_id': 'cust_123'},
            {'id': 2, 'type': 'payment', 'amount': 99.99, 'customer_id': 'cust_123'},
            {'id': 3, 'type': 'shipping', 'order_id': 1, 'tracking': 'TRK123'}
        ]
    }
    response = faas_mgr.invoke(stream_task.name, test_events, provider='nuclio-aws')
    print(f"Stream processing result: {json.dumps(response, indent=2)}")
except Exception as e:
    print(f"Stream processor deployment failed: {e}")
    traceback.print_exc()

## 3. Container Image Deployment from ECR

In [ ]:
# Deploy from existing ECR image
# First, let's create a simple function to build an image
build_task = Task()
build_task.provider = 'nuclio'
build_task.cloud_provider = 'aws'
build_task.memory = 1024
build_task.vcpus = 0.5

# This will build and push to ECR
build_task.runtime = 'python:3.9'
build_task.handler = 'api:serve'
build_task.handler_code = '''
import json
import os

def serve(context, event):
    path = event.path or '/'
    method = event.method or 'GET'
    
    response = {
        'path': path,
        'method': method,
        'message': 'API endpoint ready',
        'version': '1.0.0'
    }
    
    return json.dumps(response)
'''

try:
    print("Building and deploying API function...")
    faas_mgr.submit(build_task)
    result = build_task.result()
    print(f"API function deployed: {result}")
    
    # Now you have an image in ECR that you can reference
    # The image URI would be something like:
    # {account}.dkr.ecr.{region}.amazonaws.com/nuclio-functions:processor-{function-name}-latest
    
except Exception as e:
    print(f"Build deployment failed: {e}")
    traceback.print_exc()

In [ ]:
# Deploy from pre-existing ECR image (example)
# container_task = Task()
# container_task.provider = 'nuclio'
# container_task.cloud_provider = 'aws'
# container_task.memory = 4096
# container_task.vcpus = 2
# 
# # Replace with your actual ECR image URI
# container_task.image = '{account}.dkr.ecr.{region}.amazonaws.com/ml-model:latest'
# 
# # Add attributes
# container_task.handler = 'handler:predict'
# container_task.runtime = 'python:3.9'
# container_task.min_replicas = 1
# container_task.max_replicas = 20
# 
# # Uncomment to deploy (requires valid ECR image)
# try:
#     faas_mgr.submit(container_task)
#     result = container_task.result()
#     print(f"Container function deployed: {result}")
# except Exception as e:
#     print(f"Container deployment failed: {e}")
#     traceback.print_exc()

## Advanced Features

In [ ]:
# Batch deployment for microservices
services = []
service_names = ['auth', 'data', 'api']

for name in service_names:
    task = Task()
    task.provider = 'nuclio'
    task.cloud_provider = 'aws'
    task.memory = 512
    task.vcpus = 0.5
    
    # Add attributes
    task.runtime = 'python:3.9'
    task.handler = f'{name}:handle'
    task.handler_code = f'''
import json
import os

def handle(context, event):
    service_name = '{name}'
    return json.dumps({{
        'service': service_name,
        'status': 'healthy',
        'instance': os.environ.get('HOSTNAME', 'unknown'),
        'message': f'{{service_name}} service is running'
    }})
'''
    task.env_vars = {
        'SERVICE_NAME': name,
        'ENVIRONMENT': 'production'
    }
    task.min_replicas = 2
    task.max_replicas = 10
    services.append(task)

try:
    # Deploy microservices
    print("Deploying microservices...")
    faas_mgr.submit(services)
    for task, name in zip(services, service_names):
        result = task.result()
        print(f"{name} service deployed: {task.name}")
except Exception as e:
    print(f"Batch deployment failed: {e}")
    traceback.print_exc()

In [ ]:
# High-performance configuration with GPU support (conceptual)
gpu_task = Task()
gpu_task.provider = 'nuclio'
gpu_task.cloud_provider = 'aws'
gpu_task.memory = 8192
gpu_task.vcpus = 4

# Add attributes
gpu_task.runtime = 'python:3.9'
gpu_task.handler = 'gpu:process'
gpu_task.handler_code = '''
import json
import torch

def process(context, event):
    # Check if CUDA is available
    cuda_available = torch.cuda.is_available()
    device_count = torch.cuda.device_count() if cuda_available else 0
    
    return json.dumps({
        'cuda_available': cuda_available,
        'device_count': device_count,
        'message': 'GPU function ready' if cuda_available else 'Running on CPU'
    })
'''

# Add resource requirements (conceptual - would need GPU node pool)
gpu_task.resources = {
    'requests': {
        'nvidia.com/gpu': '1'  # Request 1 GPU
    },
    'limits': {
        'nvidia.com/gpu': '1'
    }
}

gpu_task.build_commands = [
    'pip install torch --index-url https://download.pytorch.org/whl/cu118'
]

# Uncomment if you have GPU nodes in your EKS cluster
# try:
#     print("Deploying GPU function...")
#     faas_mgr.submit(gpu_task)
#     result = gpu_task.result()
#     print(f"GPU function deployed: {result}")
# except Exception as e:
#     print(f"GPU deployment failed: {e}")
#     traceback.print_exc()

In [ ]:
# List and manage functions
try:
    functions = faas_mgr.list_functions()
    print("Deployed functions:")
    for provider, funcs in functions.items():
        print(f"\nProvider: {provider}")
        for func in funcs:
            print(f"  - {func['name']} (state: {func.get('state', 'unknown')}, replicas: {func.get('replicas', 0)})")
            print(f"    URL: {func.get('url', 'N/A')}")

    # Get manager status
    status = faas_mgr.get_status()
    print(f"\nManager status: {json.dumps(status, indent=2)}")
    
    # Get provider health
    health = faas_mgr.get_provider_health()
    print(f"\nProvider health: {json.dumps(health, indent=2)}")
except Exception as e:
    print(f"Failed to list functions: {e}")
    traceback.print_exc()

## Cleanup

In [ ]:
# Delete specific function
# try:
#     if 'inline_task' in locals() and hasattr(inline_task, 'name'):
#         faas_mgr.delete_function(inline_task.name, provider='nuclio')
#         print(f"Deleted function: {inline_task.name}")
# except Exception as e:
#     print(f"Failed to delete function: {e}")

# Shutdown manager (auto_terminate=True will cleanup EKS cluster)
# WARNING: This will delete the EKS cluster and all resources!
# try:
#     print("Shutting down FaaS manager and cleaning up EKS cluster...")
#     faas_mgr.shutdown()
#     print("Shutdown complete")
# except Exception as e:
#     print(f"Failed to shutdown manager: {e}")